In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, f1_score, fbeta_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier

In [76]:
fault_diag = pd.read_csv('../data/fault_diag.csv')

/var/folders/xm/vmtk5pms0jzb2kmtm00lwtn80000gn/T/ipykernel_54025/699357233.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  fault_diag = pd.read_csv('../data/fault_diag.csv')


In [77]:
fault_diag.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi,fuel_gas_ratio,oil_cool_temp_diff
0,1038243,55748536,2018-07-20 09:31:33,High (Severity Medium) J1939 Network #2,49,1231,16,True,2,105406655,...,NaN,True,138.2,255.0,5.203984,100.0,5.80,1231_16,0.082813,22.33120
1,366301,7171498,2016-01-31 07:12:25,NaN,11,629,12,True,127,105301976,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,629_12,NaN,NaN
2,1141606,81618595,2019-03-27 08:10:52,Low (Severity Medium) Engine Coolant Level,0,111,18,True,1,105338729,...,NaN,True,59.0,2047.0,0.000000,100.0,0.29,111_18,0.073593,-13.89375
3,1142275,81843663,2019-03-29 06:08:56,Low (Severity Medium) Engine Coolant Level,0,111,18,True,3,105338729,...,NaN,True,73.4,2047.0,0.000000,100.0,0.00,111_18,0.069011,-2.41875
4,1149647,83749925,2019-04-16 12:27:33,Low (Severity Medium) Engine Coolant Level,0,111,18,True,4,105338729,...,NaN,True,95.0,2047.0,67.525570,100.0,3.48,111_18,0.002277,46.80000


In [78]:
# sort by EventTimeStamp
fault_diag_sorted = fault_diag.sort_values('EventTimeStamp').reset_index(drop = True)

In [79]:
fault_diag_sorted.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi,fuel_gas_ratio,oil_cool_temp_diff
0,1211418,108604426,2000-03-18 19:14:10,High Voltage (Fuel Level),49,96,3,True,126,2015,...,NaN,True,127.4,1279.0,0.000000,100.0,0.58,96_3,0.044887,7.6500
1,1211417,108604425,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),49,829,3,True,126,2015,...,NaN,True,127.4,1279.0,0.000000,100.0,0.58,829_3,0.044887,7.6500
2,1211423,108608954,2000-03-19 03:58:23,Not Reporting Data Wheel Sensor ABS Axle 2 Right,11,792,7,True,14,1849,...,32.0,True,82.4,1279.0,46.641670,100.0,14.50,792_7,0.073241,2.5875
3,1211432,108615304,2000-03-19 07:32:53,Low (Severity Medium) Transmission Air Tank Pr...,3,37,18,True,19,2283,...,NaN,True,118.4,50175.0,0.145634,100.0,0.87,37_18,0.067791,19.5750
4,1211436,108620730,2000-03-19 08:40:03,High Voltage (Fuel Level),49,96,3,True,126,2034,...,NaN,True,116.6,1279.0,0.000000,100.0,0.00,96_3,0.036621,21.2625


In [80]:
# to find the time at which the 80% event happens, use loc to get just derate rows
derates = fault_diag_sorted.loc[(fault_diag_sorted['spn'] == 5246)].reset_index(drop = True)
# len * .8 to find how many events to include in train
ts_08 = int(len(derates) * 0.8)
ts_06 = int(len(derates) * 0.6)
# get timestamp corresponding to that row number, everything at or before is train
split_08 = derates.loc[ts_08, 'EventTimeStamp']
split_06 = derates.loc[ts_06, 'EventTimeStamp']

In [74]:
# X_train
# fault_diag_sorted.loc[fault_diag_sorted['EventTimeStamp'] <= split_06]

#X_val
# fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_06) & (fault_diag_sorted['EventTimeStamp'] <= split_08)]

# X-test
# fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_08)]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,...,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,Speed,Throttle,TurboBoostPressure,spn_fmi
333841,718354,14647419,2017-02-21 08:14:34,Low (Severity Low) Engine Coolant Level,0,111,17,True,126,1816,...,26024.777617,2.826650,32.0,True,77.0,17407.0,8.398220,100.0,2.90,111_17
333842,718353,14647409,2017-02-21 08:15:46,Low (Severity Low) Catalyst Tank Level,0,1761,17,True,1,1608,...,67268.507149,18.241140,NaN,True,96.8,1023.0,66.040110,100.0,24.94,1761_17
333843,718358,14647521,2017-02-21 08:20:29,Low (Severity Low) Engine Coolant Level,0,111,17,True,1,1564,...,78039.990484,14.833310,NaN,True,86.0,1023.0,42.942570,NaN,27.26,111_17
333844,718361,14647562,2017-02-21 08:21:21,Low (Severity Low) Engine Coolant Level,0,111,17,True,126,1807,...,28503.371895,0.000000,32.0,True,109.4,1023.0,34.622020,100.0,0.29,111_17
333845,718386,14648083,2017-02-21 08:21:51,Abnormal Update Rate Tire Location,49,929,9,True,126,1646,...,73362.824303,0.792519,32.0,True,145.4,1279.0,1.097108,100.0,0.58,929_9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465867,1029744,53168368,2018-06-25 14:07:16,High Voltage (Fuel Level),49,96,3,True,126,1921,...,29712.883635,9.325302,NaN,True,131.0,1279.0,66.807110,100.0,11.31,96_3
465868,1029748,53170856,2018-06-25 14:27:22,Low (Severity Low) Engine Coolant Level,0,111,17,True,126,1806,...,44172.605073,11.385850,32.0,True,109.4,1023.0,33.262780,100.0,7.25,111_17
465869,1029756,53172424,2018-06-25 14:45:22,Incorrect Data J1939 Network #1 Primary Vehicl...,11,639,2,True,127,1733,...,76682.410308,4.847572,NaN,True,152.6,1279.0,4.699120,100.0,2.61,639_2
465870,1029757,53172740,2018-06-25 14:47:46,Special Instructions Maximum Vehicle Speed Limit,0,74,14,True,1,305,...,43387.353648,NaN,NaN,True,134.6,22527.0,NaN,NaN,NaN,74_14


Defining functions for determining optimal threshold and applying it to test set.

In [93]:
# find optimal threshold
def find_optimal_threshold(y_val, y_proba):
    optimal_threshold = 0
    max_savings = 0

    # test threshold values to find the best
    for threshold in np.arange(0, 1, 0.001):

        y_pred = y_proba > threshold

        # create confusion matrix
        conf_matrix = confusion_matrix(y_val, y_pred)

        tp = conf_matrix[1, 1]
        fp = conf_matrix[0, 1]

        # savings calculated
        savings = (4000 * tp) - (500 * fp)

        # check to see if it's best
        if savings > max_savings:
            max_savings = savings
            optimal_threshold = threshold
    
    return optimal_threshold

In [97]:
# apply optimal threshold to test data
def pred_with_optimal_threshold(y_test, y_proba, threshold):
    
    y_pred = (y_proba >= threshold).astype(int)

    return y_pred

# y_pred_adjusted = (y_proba >= threshold).astype(int)

In [106]:
# calculate and print metrics
def calculate_metrics(y_test, y_pred):
    
    metrics = {}
    
    metrics['accuracy'] = accuracy_score(y_test, y_pred)
    metrics['matthews_corrcoef'] = matthews_corrcoef(y_test, y_pred)
    metrics['classification_report'] = classification_report(y_test, y_pred, zero_division = 0.0)
    metrics['confusion_matrix'] = confusion_matrix(y_test, y_pred)
    metrics['model_savings'] = metrics['confusion_matrix'][1, 1] * 4000 - metrics['confusion_matrix'][0, 1] * 500
    
    return metrics

def print_metrics(metrics):
    
    print(f'Accuracy score: {metrics["accuracy"]}')
    print(f'Matthews corr coef: {metrics["matthews_corrcoef"]}')
    print('Classification Report:')
    print(metrics['classification_report'])
    print('Confusion Matrix:')
    print(metrics['confusion_matrix'])
    print(f'Model savings: {metrics["model_savings"]}')

In [107]:
# modifying previously used model to use time-based split

# to find the time at which the 80% event happens, use loc to get just derate rows
derates = fault_diag_sorted.loc[(fault_diag_sorted['spn'] == 5246)].reset_index(drop = True)
# len * .8 to find how many events to include in train
ts_08 = int(len(derates) * 0.8)
ts_06 = int(len(derates) * 0.6)
# get timestamp corresponding to that row number, everything at or before is train
split_08 = derates.loc[ts_08, 'EventTimeStamp']
split_06 = derates.loc[ts_06, 'EventTimeStamp']

# sets up split (still has all columns, could go ahead and select columns here next time)
train_set = fault_diag_sorted.loc[fault_diag_sorted['EventTimeStamp'] <= split_06]
val_set = fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_06) & (fault_diag_sorted['EventTimeStamp'] <= split_08)]
test_set = fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_08)]


# take AcceleratorPedal, FuelLtd, DistanceLtd, FuelRate, Speed out of model and try it with cost threshold
variables = ['BarometricPressure', 'CruiseControlSetSpeed', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 
             'FuelLevel', 'IntakeManifoldTemperature', 'TurboBoostPressure', 'FuelTemperature', 'Throttle']
target = 'derate_full'

# replaces train_test_split
X_train = train_set[variables].copy()
X_val = val_set[variables].copy()
X_test = test_set[variables].copy()

y_train = train_set[target].copy()
y_val = val_set[target].copy()
y_test = test_set[target].copy()

# same model as before
# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy = 'mean')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)
X_test_imputed = imputer.transform(X_test)

logreg = LogisticRegression(max_iter = 1000).fit(X_train_imputed, y_train)

# predicted probabilities for validation data
y_proba_val = logreg.predict_proba(X_val_imputed)[:, 1]

optimal_threshold = find_optimal_threshold(y_val, y_proba_val)
print(optimal_threshold)

# not sure this is actually how I should do this, but using the y_proba from the validation set was failing because there are different numbers of rows
y_proba_test = logreg.predict_proba(X_test_imputed)[:, 1]
y_pred_test = pred_with_optimal_threshold(y_test, y_proba_test, optimal_threshold)

metrics = calculate_metrics(y_test, y_pred_test)
print_metrics(metrics)

0.244
Accuracy score: 0.9932488042272505
Matthews corr coef: 0.07089068333507767
Classification Report:
              precision    recall  f1-score   support

       False       0.99      1.00      1.00     83281
        True       0.32      0.02      0.03       556

    accuracy                           0.99     83837
   macro avg       0.66      0.51      0.51     83837
weighted avg       0.99      0.99      0.99     83837

Confusion Matrix:
[[83262    19]
 [  547     9]]
Model savings: 26500


calculate cost if there was no model (so 2nd column in confusion matrix is 0, 0)
- 4000 * num derates in data

comparison:
- 4000 for each missed (lower left) + cost for false positive (upper right)
- difference is amount saved

quicker: just look at right side (calculate net benefit)
- saved 4000 for upper right and cost 500 for lower right
- upper right increases cost
- lower right decreases cost
- positive number = better than no model, negative means worse

Trying hist gradient boosting classifier - suggestion from Michael
- tree-based classifier
- native support for missing values so try dropping the imputation I've been using

In [110]:
# repeated from previous model - set up custom-defined split based on derate events
# to find the time at which the 80% event happens, use loc to get just derate rows
derates = fault_diag_sorted.loc[(fault_diag_sorted['spn'] == 5246)].reset_index(drop = True)
# len * .8 to find how many events to include in train
ts_08 = int(len(derates) * 0.8)
ts_06 = int(len(derates) * 0.6)
# get timestamp corresponding to that row number, everything at or before is train
split_08 = derates.loc[ts_08, 'EventTimeStamp']
split_06 = derates.loc[ts_06, 'EventTimeStamp']

# sets up split (still has all columns, could go ahead and select columns here next time)
train_set = fault_diag_sorted.loc[fault_diag_sorted['EventTimeStamp'] <= split_06]
val_set = fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_06) & (fault_diag_sorted['EventTimeStamp'] <= split_08)]
test_set = fault_diag_sorted.loc[(fault_diag_sorted['EventTimeStamp'] > split_08)]


# take AcceleratorPedal, FuelLtd, DistanceLtd, FuelRate, Speed out of model and try it with cost threshold
variables = ['BarometricPressure', 'CruiseControlSetSpeed', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 
             'FuelLevel', 'IntakeManifoldTemperature', 'TurboBoostPressure', 'FuelTemperature', 'Throttle']
target = 'derate_full'

# replaces train_test_split
X_train = train_set[variables].copy()
X_val = val_set[variables].copy()
X_test = test_set[variables].copy()

y_train = train_set[target].copy()
y_val = val_set[target].copy()
y_test = test_set[target].copy()

hgbm = HistGradientBoostingClassifier().fit(X_train, y_train)

# predicted probabilities for validation data
y_proba_val = hgbm.predict_proba(X_val)[:, 1]

optimal_threshold = find_optimal_threshold(y_val, y_proba_val)
print(optimal_threshold)

# not sure this is actually how I should do this, but using the y_proba from the validation set was failing because there are different numbers of rows
y_proba_test = hgbm.predict_proba(X_test)[:, 1]
y_pred_test = pred_with_optimal_threshold(y_test, y_proba_test, optimal_threshold)

metrics = calculate_metrics(y_test, y_pred_test)
print_metrics(metrics)

# this identified everything as true, definitely need to do better

0
Accuracy score: 0.006631916695492444
Matthews corr coef: 0.0
Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00     83281
        True       0.01      1.00      0.01       556

    accuracy                           0.01     83837
   macro avg       0.00      0.50      0.01     83837
weighted avg       0.00      0.01      0.00     83837

Confusion Matrix:
[[    0 83281]
 [    0   556]]
Model savings: -39416500
